# December 14, 2022
https://adventofcode.com/2022/day/14

In [1]:
fn = "data/14.txt"
with open(fn, "r") as file:
    lines = file.readlines()
puz = [line.strip("\n") for line in lines]

In [2]:
test = ["498,4 -> 498,6 -> 496,6", "503,4 -> 502,4 -> 502,9 -> 494,9"]

In [8]:
import numpy as np

class Regolith:  
    def __init__( self, rock_array, floor = False ):
        # floor = True is for Part 2
        tmp = rock_array[0].split(" -> ")[0].split(",")
        self.minx = self.maxx = int(tmp[0])
        self.miny = 0
        self.maxy = int(tmp[1])
        self.array = np.array( [["."] * (self.maxy+1)] ).reshape([self.maxy+1,1])

        for line in rock_array:
           # print(line)
            pairs = [ [int(n) for n in pair.split(",")] for pair in line.split(" -> ") ]
            for p0, p1 in zip( pairs[:-1], pairs[1:] ):
                self.add_rock( p0, p1 )
            
        self.sand_count = 0
        self.floor = floor

    def add_rock( self, p0, p1 ):
        # make sure there's room
        x0 = min(p0[0], p1[0])
        x1 = max(p0[0], p1[0])
        y0 = min(p0[1], p1[1])
        y1 = max(p0[1], p1[1])
        self.expand( [x0,x1], [y0,y1] )
       # print("\n", self.array)

        x0 = self.getx(x0)
        x1 = self.getx(x1)
        y0 = self.gety(y0)
        y1 = self.gety(y1)
        self.array[y0:y1+1, x0:x1+1] = "#"
       # print("\n", self.array)


    # functions to convert absolute index to the array map
    def getx( self, x ):
        return x - self.minx
    def gety( self, y ):
        return y - self.miny

    # add extra space as needed to array
    def expand( self, xlim, ylim ):
        if xlim[0] < self.minx:
            dx = self.minx - xlim[0]
            h = self.maxy - self.miny + 1
            space = np.array( ["."]*dx*h ).reshape(h, dx )
            self.array = np.concatenate( (space, self.array), axis=1 )
            self.minx = xlim[0]
        if xlim[1] > self.maxx:
            dx = xlim[1] - self.maxx
            h = self.maxy - self.miny + 1
            space = np.array( ["."]*dx*h ).reshape(h, dx )
            self.array = np.concatenate( (self.array, space), axis=1 )
            self.maxx = xlim[1]
        if ylim[0] < self.miny:
            dy = self.miny - ylim[0]
            w = self.maxx - self.minx + 1
            space = np.array( ["."]*dy*w ).reshape(dy, w )
            self.array = np.concatenate( (space, self.array), axis=0 )
            self.miny = ylim[0]
        if ylim[1] > self.maxy:
            dy = ylim[1] - self.maxy
            w = self.maxx - self.minx + 1
            space = np.array( ["."]*dy*w ).reshape(dy, w )
            self.array = np.concatenate( (self.array, space), axis=0 )
            self.maxy = ylim[1]
    
    def drop_sand( self, x, y=0 ):
        x = self.getx(x)
        y = self.gety(y)
        yinit = y
        while y < self.array.shape[0] - 1:

            if self.floor:
                pad = 10
                # add horizontal floor as needed
                if x <= 0:
                    self.expand( [self.minx-pad, self.maxx], [self.miny, self.maxy] )
                    self.add_rock( [self.minx, self.maxy], [self.minx+10, self.maxy] )
                    x += pad # true coord for x is now shifted to right due to padding
                elif x >= self.array.shape[1]-2:
                    self.expand( [self.maxx, self.maxx+pad], [self.miny, self.maxy] )
                    self.add_rock( [self.maxx-pad, self.maxy], [self.maxx, self.maxy] )

            if self.array[y+1, x] == ".":
                # fall down
                y += 1
            elif self.array[y+1, x-1] == ".":
                y += 1
                x -= 1
            elif self.array[y+1, x+1] == ".":
                y += 1
                x += 1
            elif y == yinit: # sand didn't fall at all!
                self.array[y,x] = "X"
                self.sand_count += 1
                return False
            else:
                self.array[y, x] = "o"
                self.sand_count += 1
                return True
            
        return False

    def __str__( self ):
        viz = ""
        for i in range(self.array.shape[0]):
            viz += "\n" + "".join( self.array[i] ) 
        return viz

In [9]:
r = Regolith( test )

In [10]:
print(r)


..........
..........
..........
..........
....#...##
....#...#.
..###...#.
........#.
........#.
#########.


In [11]:
while r.drop_sand(x=500):
    pass
print(r.sand_count)
print(r)

24

..........
..........
......o...
.....ooo..
....#ooo##
...o#ooo#.
..###ooo#.
....oooo#.
.o.ooooo#.
#########.


In [12]:
r = Regolith(puz)
print(r)


..........................................................
..........................................................
..........................................................
..........................................................
..........................................................
..........................................................
..........................................................
..........................................................
..........................................................
..........................................................
..........................................................
..........................................................
..........................................................
..........................................................
..........................................................
.................#...#....................................
.................#...#.................................

In [87]:
while r.drop_sand(x=500):
    pass
print(r.sand_count)
print(r)

674

..........................................................
..........................................................
..........................................................
..........................................................
..........................................................
..........................................................
..........................................................
..........................................................
..........................................................
..........................................................
..........................................................
..........................................................
..........................................................
..........................................................
...................oo.....................................
.................#ooo#....................................
.................#ooo#.............................

### Part 2

In [18]:
r = Regolith(test, floor = True )
r.add_rock( [r.minx, r.maxy+2], [r.maxx, r.maxy+2] )
print(r)


..........
..........
..........
..........
....#...##
....#...#.
..###...#.
........#.
........#.
#########.
..........
##########


In [21]:
while r.drop_sand(x=500):
    pass
print(r.sand_count)
print(r)

93

................X.............
...............ooo............
..............ooooo...........
.............ooooooo..........
............oo#ooo##o.........
...........ooo#ooo#ooo........
..........oo###ooo#oooo.......
.........oooo.oooo#ooooo......
........oooooooooo#oooooo.....
.......ooo#########ooooooo....
......ooooo.......ooooooooo...
##############################


In [22]:
r = Regolith(puz, floor = True )
r.add_rock( [r.minx, r.maxy+2], [r.maxx, r.maxy+2] )
print(r)


..........................................................
..........................................................
..........................................................
..........................................................
..........................................................
..........................................................
..........................................................
..........................................................
..........................................................
..........................................................
..........................................................
..........................................................
..........................................................
..........................................................
..........................................................
.................#...#....................................
.................#...#.................................

In [23]:
while r.drop_sand(x=500):
    pass
print(r.sand_count)
print(r)

24958

..........................................................................................................................................................................X.......................................................................................................................................................................
.........................................................................................................................................................................ooo......................................................................................................................................................................
........................................................................................................................................................................ooooo..............................................................................................................................................